# HuggingFaceH4/CodeAlpaca_20K Python split

In [4]:
import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
from datasets import  load_dataset

h4ca20k = load_dataset("HuggingFaceH4/CodeAlpaca_20K")

d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(h4ca20k)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 18019
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2003
    })
})


In [11]:
print(h4ca20k["train"])

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 18019
})


In [14]:
from datasets import Dataset, DatasetDict, concatenate_datasets

In [40]:
from datasets import Features, Value

In [39]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="") 

In [23]:
def is_py(prompts):
    py_dataset = Dataset.from_dict({})
    for i in prompts:

        response = client.chat.completions.create(
            model="glm-4", 
    
            messages=[


                {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
                {"role": "user", "content": i["prompt"]},
                
        
            ],
        )

        if(response.choices[0].message.content == "1"):
            py_dataset.add_item(i)

    return py_dataset


In [26]:
prompts  = h4ca20k["test"]

In [56]:

features = Features({
    'prompt': Value('string'),
    'completion': Value('string')
})


py_dataset = Dataset.from_dict({'prompt':[], 'completion':[]},features=features)

In [57]:

cot = 0
for i in prompts:

    response = client.chat.completions.create(
        model="glm-4", 
    
        messages=[


            {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
            {"role": "user", "content": i["prompt"]},
                
        
        ],
    )

    if(response.choices[0].message.content == "1"):
        
        cot = cot+1
        print(cot)

        py_dataset = concatenate_datasets([py_dataset,Dataset.from_dict({'prompt':[i['prompt']],'completion':[i['completion']]})])
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [58]:
py_test = py_dataset

In [59]:
print(py_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 548
})


In [ ]:
def filter_entries(example):
    if is_python_related(example['prompt']):
        return example

filtered_dataset = dataset.map(filter_entries, batched=True)


In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Convert the filtered dataset to a PyArrow Table
table = pa.Table.from_pandas(filtered_dataset.to_pandas())

# Save the table to a Parquet file
pq.write_table(table, 'filtered_dataset.parquet')
